<span style="display: inline-block; margin-right: 10px; vertical-align: middle; border: 1px solid #F8F8F8; border-radius: 8px; padding: 5px; background-color: #F8F8F8;">
    <a href="https://www.designsafe-ci.org/workspace/jupyter-lab-hpc-cuda-ds?appVersion=1.1.1" target="_parent" style="text-decoration: none;">
        <span style="font-family: Helvetica, sans-serif; font-size: 13px; color: #565656; margin-right: 5px; vertical-align: middle; font-weight: 600;">Open in</span>
        <img src="https://www.designsafe-ci.org/media/filer_public/2d/d3/2dd37fbf-289e-49cf-9c1a-879c864c4e17/nsf_nheri-ds.png" alt="Open in DesignSafe" style="width: 100px; height: auto; vertical-align: middle;">
    </a>
</span>

<span style="display: inline-block; margin-right: 10px; vertical-align: middle;">
    <a href="https://colab.research.google.com/github/NHERI-SimCenter/BrailsPlusPlus/blob/master/examples/image_processor/occupancy_classifier.ipynb" target="_parent">
        <img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab" style="width: 130px; height: auto; vertical-align: middle;"/>
    </a>
</span>

<span style="display: inline-block; vertical-align: middle; margin-top: 6px;">
    <a target="_blank" href="https://lightning.ai/new?repo_url=https%3A//github.com/NHERI-SimCenter/BrailsPlusPlus/blob/master/examples/image_processor/occupancy_classifier.ipynb">
        <img src="https://pl-bolts-doc-images.s3.us-east-2.amazonaws.com/app-2/studio-badge.svg" alt="Open in Studio" style="width: 120px; height: auto; vertical-align: middle;"/>
    </a>
</span>

## **Install BRAILS++**
BRAILS++ is not yet available on PyPi. The following line installs the latest version from the GitHub repository using `pip`.

In [1]:
!pip install git+https://github.com/NHERI-SimCenter/BrailsPlusPlus

  Cloning https://github.com/bacetiner/BrailsPlusPlus to /tmp/pip-req-build-a7j8w_kr
  Running command git clone --filter=blob:none --quiet https://github.com/bacetiner/BrailsPlusPlus /tmp/pip-req-build-a7j8w_kr
  Resolved https://github.com/bacetiner/BrailsPlusPlus to commit 254ceb02fd598d141f14982d8f09d13af3a3990e
  Preparing metadata (setup.py) ... done
  Created wheel for brails: filename=brails-4.0-py3-none-any.whl size=2009215 sha256=2470f4dee6e095bdfddae3dadf9d7962f6d4a617e9303af646e5df4877e22be5
  Stored in directory: /tmp/pip-ephem-wheel-cache-22ckp2bz/wheels/65/0b/aa/a9113e0557b48ca6dfe52e54ff2df5af4aa9945a3736b53797
Successfully built brails


## **Import ImageSet and Importer Methods of BRAILS++**
BRAILS++ offers various Artificial Intelligence-powered modules for predicting building and infrastructure attributes from images, such as the `OccupancyClassifier` demonstrated in this example.

The `Importer` class provides the main methods to load these modules. The `ImageSet` class and BRAILS++ scrapers dedicated to this purpose can be utilized to get the images needed to predict attributes. This example will illustrate both approaches.

In [2]:
from brails.types.image_set import ImageSet
from brails import Importer

## **Define Google API Key**
You need a Google API Key with Street View Static API enabled to download the Google Street View imagery required for this example. Please follow this [link](https://support.google.com/googleapi/answer/6158862?hl=en) for instructions on setting up a Google API key.

In [3]:
API_KEY = 'YOUR-API-KEY-HERE'

## **Load Street-Level Imagery: (Option 1) Download Imagery by Specifying the Region Name**
The `OccupancyClassifier` class is created to identify occupancy types of buildings using street-level imagery. In this option, BRAILS++ retrieves the required imagery by following the steps below.

1. <strong>Call the `RegionBoundary` class with the necessary details to specify the region of interest.</strong> In this example, street-level imagery of buildings in Lake Charles, LA, are downloaded. The required information to download these images includes the `type` of information provided and the `data` itself, which are the `locationName` and `Lake Charles, LA`.

2. <strong>Download the footprint inventory for the `RegionBoundary` saved in `region_boundary_object` by downloading the OpenStreetMap footprint data through the `MS_FootprintScraper`.</strong> The only information required to initialize `MS_FootprintScraper` is the desired output unit for `length`, which, in this case, is `ft`. The output of `MS_FootprintScraper` is an `AssetInventory` saved in `lake_charles_fp`.

3. <strong>Get a randomly selected `50` building subset of the footprint inventory obtained using `MS_FootprintScraper`. </strong> This subset is obtained using the `get_random_sample` method of an `AssetInventory` object and is stored in `lake_charles_fp_subset`. For this example, the random seed is set to `10` for reproducibility of the results.

4. <strong>Get the street-level imagery for the selected subset using `get_images` method of `GoogleStreetview` module.</strong> `get_images` requires two inputs 1) AssetInventory for which the images will be retrieved (in this example, `lake_charles_fp_subset`) and 2) the path to the folder where the retrieved images will be saved, which in this case is `tmp/street/`.

In [4]:
# Select a region and create its RegionBoundary:
importer = Importer()

region_data = {"type": "locationName",
               "data": "Lake Charles, Louisiana"}
region_boundary_class = importer.get_class("RegionBoundary")
region_boundary_object = region_boundary_class(region_data)

usa_fp_class = importer.get_class("MS_FootprintScraper")
usa_fp_data = {"length": "ft"}
footprint_scraper = usa_fp_class(usa_fp_data)
lake_charles_fp = footprint_scraper.get_footprints(region_boundary_object)

# Subsample from the extracted assets to keep the image downloading step quick.
# Here, we are randomly sampling 50 buildings using a random seed value of 10:
lake_charles_fp_subset = lake_charles_fp.get_random_sample(50, 10)

# Get street-level imagery for the selected subset using GoogleStreetview:
google_street_class = importer.get_class("GoogleStreetview")
google_street = google_street_class({'apiKey': API_KEY})
images_street = google_street.get_images(
    lake_charles_fp_subset, 'tmp/street/')


Searching for Lake Charles, Louisiana...
Found Lake Charles, Calcasieu Parish, Louisiana, United States


Processing quadkeys: 100%|██████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████| 2/2 [00:54<00:00, 27.14s/it]



Found a total of 34913 building footprints in Lake Charles


INFO:brails.scrapers.google_streetview.google_streetview:
Images will be saved to: /home/bacetiner/Documents/BrailsPlusPlus/examples/image_processor/tmp/street

Obtaining street-level imagery:   8%|█████████▊                                                                                                                | 4/50 [00:03<00:33,  1.38it/s]INFO:brails.scrapers.google_streetview.google_streetview:No street-level imagery found for the building located at 30.2451, -93.2056
INFO:brails.scrapers.google_streetview.google_streetview:No street-level imagery found for the building located at 30.2518, -93.1727
Obtaining street-level imagery: 100%|█████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████| 50/50 [00:11<00:00,  4.53it/s]


## **Predict Building Occupancies Using OccupancyClassifier Module**
`OccupancyClassifier` includes a pre-trained EfficientNet_V2_M model that has been trained on a custom dataset of 50,000 images. This model is capable of classifying buildings visible in street-level imagery into two categories: `Residential` and `Other`.

In [5]:
importer = Importer()
occupancy_classifier_class = importer.get_class('OccupancyClassifier')
occupancy_classifier = occupancy_classifier_class()
predictions = occupancy_classifier.predict(images_street)


Default occupancy classifier model at tmp/models/OccupancyClassifier_v1.pth loaded


/home/bacetiner/anaconda3/envs/brails_plus/lib/python3.11/site-packages/torch/cuda/__init__.py:128: UserWarning: CUDA initialization: CUDA unknown error - this may be due to an incorrectly set up environment, e.g. changing env variable CUDA_VISIBLE_DEVICES after program start. Setting the available devices to be zero. (Triggered internally at ../c10/cuda/CUDAFunctions.cpp:108.)
  return torch._C._cuda_getDeviceCount() > 0
/home/bacetiner/anaconda3/envs/brails_plus/lib/python3.11/site-packages/brails/processors/image_classifier/image_classifier.py:759: FutureWarning: You are using `torch.load` with `weights_only=False` (the current default value), which uses the default pickle module implicitly. It is possible to construct malicious pickle data which will execute arbitrary code during unpickling (See https://github.com/pytorch/pytorch/blob/main/SECURITY.md#untrusted-models for more details). In a future release, the default value for `weights_only` will be flipped to `True`. This limits

## **Show Prediction Results**

In [6]:
print(predictions)

{2135: 'Residential', 28107: 'Residential', 31625: 'Residential', 972: 'Residential', 13506: 'Other', 30315: 'Residential', 32197: 'Residential', 18187: 'Other', 10500: 'Residential', 2254: 'Residential', 34122: 'Residential', 32118: 'Residential', 21480: 'Other', 4986: 'Residential', 16383: 'Other', 23668: 'Residential', 2920: None, 27551: 'Residential', 9089: 'Residential', 23270: 'Residential', 25008: 'Other', 27607: 'Residential', 18587: None, 17191: 'Residential', 29938: None, 11447: 'Other', 19864: 'Residential', 23762: 'Residential', 8714: 'Residential', 29945: 'Other', 15688: 'Residential', 28812: 'Other', 24591: 'Other', 2900: 'Residential', 265: 'Residential', 15443: 'Residential', 8780: 'Residential', 12776: 'Residential', 19848: 'Residential', 23995: 'Residential', 15740: 'Residential', 20602: 'Residential', 29532: 'Other', 28575: 'Residential', 30800: 'Other', 4259: 'Residential', 21278: 'Residential', 32893: 'Residential', 10251: 'Residential', 14699: 'Other'}
